In [1]:
import numpy as np
import os
import itertools as it
from scipy.spatial.distance import pdist, squareform
from numba import jit
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="white", color_codes=True)

In [2]:
# import inference code
from dbf_testing import dbf_test
# import prediction code
from sklearn import neighbors
from sklearn import grid_search

knn_error_metric = "accuracy"
knn_p = 1
knn_parameters = {"weights":('uniform', 'distance'), "p":(knn_p,)}    
knn = neighbors.KNeighborsClassifier()
# parameters = {'n_neighbors': np.logspace(np.log10(1), np.log10(len(y_sub)/2), 10, dtype=int), "weights":('uniform', 'distance'), "p":(1, 2)}
# grid = grid_search.GridSearchCV(knn, parameters, cv=ncv, scoring=error_metric)


C:\Users\Peter\Anaconda3\envs\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Peter\Anaconda3\envs\python27\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
import glob
import pandas as pd #panda library

In [4]:
# loading data function
def load_idxlist(datadir):
    '''
    idxlist = list of loaded subject ids
    '''
    idxlist = [] # This is a list 
    
    for fh in sorted(glob.glob(datadir+"\\*.txt")): # look for files like sub01.txt
        sub = int(((fh.split("\\")[-1]).split(".")[0]).lstrip('sub')) # get subject ID
        idxlist.append(sub)
        
    return idxlist

In [137]:
def load_distmatrix(infile):
    X = np.loadtxt(infile, delimiter = ",")
    X = X + np.transpose(X)
    return X

In [138]:
indir = os.path.join("C:\Users", "Peter", "Dropbox", "Documents(newlaptop)", "Neuroscience", "cutnorm", "FOptM", "FOptM-share")
idx = 10
infile = os.path.join(indir, "%s%02d_lw.csv"%("corrdist_0.", idx,))
print infile
distmatrix = load_distmatrix(infile)

C:\Users\Peter\Dropbox\Documents(newlaptop)\Neuroscience\cutnorm\FOptM\FOptM-share\corrdist_0.10_lw.csv


In [7]:
def load_behav_data(behaviorfile, newidx):
    # load the behavior files
    behaviors = pd.io.parsers.read_csv(behaviorfile, sep='\t', na_values=".")
    # only look at behaviors whose networks we've analyzed
    behaviors['idx'] = behaviors['Isubcode']
    for (i,j) in enumerate(behaviors['idx']):
        behaviors['idx'][i] = int(behaviors['idx'][i].lstrip('sub'))

    val_mask = [0]*len(behaviors['idx'])
    for (i,idx) in enumerate(behaviors['idx']):
        val_mask[i] = (idx in newidx)
        
    behaviors = behaviors.loc[val_mask]
    
    # convert to dictionary
    outdata = behaviors.to_dict(orient='list')
    for key, val in outdata.iteritems():
        outdata[key] = np.array(val)
    
    return outdata

In [8]:
datadir = os.path.join("C:\Users", "Peter", "Dropbox", "Documents(newlaptop)", "Neuroscience", "cutnorm", "FOptM", "FOptM-share", "data", "updated")
idxlist = load_idxlist(datadir)

In [9]:
behaviorfile = "trackingdata.txt"
targets = load_behav_data(behaviorfile, idxlist)

C:\Users\Peter\Anaconda3\envs\python27\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
def preprocessor(ydict, kmax=5, nfrac=.10, amin = 5):
    N = len(ydict)
    for kx, vx in ydict.iteritems():
        K = len(vx)
        break
    
    selected = dict()
    outydict = dict()
    outbins  = dict()
    for idx, (ykey, yval) in enumerate(ydict.iteritems()):
        
        if len(yval)<10:
            continue
        
        #print ykey, yval[:10]
        
        # remove Nans
        try:
            topselect = np.logical_not(np.isnan(yval))
        except: # probably string. Skip and let others handle it
            topselect = np.ones(len(yval), dtype=bool)
            #print "continue-NaN"
            #continue
            
        ymasked = yval[topselect]
        # check how may bins you have
        yuniq, yinv, ycount = np.unique(ymasked, return_inverse=True, return_counts=True)
        
        # check if discrete or continuous
        bins = None
        if len(ycount) > len(ymasked)/2: # likely contunous variables
            try:
                ymasked = np.array(ymasked, dtype=float) # double check
            except: # skip this. Its is not convertible, but takes too many values for discrete
                continue 
            
            # reset he values based on histogram
            _, bins = np.histogram(ymasked, bins=kmax)
            ymasked = np.digitize(ymasked, bins, right=True)
                    
            # re-initialize unique count using digitized results
            yuniq, yinv, ycount = np.unique(ymasked, return_inverse=True, return_counts=True)
        
        
        yselect = np.ones(shape=(len(ycount),), dtype=bool) # sub-selector using counts
        
        # remove elements with less than absolute min count
        yselect[ycount<amin] = False
        
        # determine threshold count, and remove bins with too few
        nmin = .25*ycount.max()
        yselect[ycount<nmin] = False
        
        # select up to kmax of the remaining
        temp = np.sort(ycount[yselect])[::-1]
        if len(temp) > kmax:
            yselect[ycount<temp[kmax]] = False
            # prune
        # else, do-nothing
        
        # check if more than two groups remain
        if len(ycount[yselect]) >= 2:
            idxselect = yselect[yinv]
            
            # print len(topselect), topselect.sum(), len(idxselect), idxselect.sum()
            selected[ykey] = topselect
            selected[ykey][topselect] = idxselect # selected index
            
            outydict[ykey] = yinv[idxselect] # binned version of Y
            outbins[ykey] = bins
        else:
            #print "continue-end"
            continue
                    
    return selected, outydict, outbins


In [129]:
# compute p value for each variable
def inferencePrediction(distance_matrix, targets, ncv=5, kmax=5, nfrac=.10, amin=5):
    ''' computes p values and CV score for each target used as a grouping
    '''
    keylist = []
    pvals = []
    scores = []
    baseline = []
    
    # distance_matrix preloaded
    
    # compute vallid y's
    selected, subYdict = preprocessor(targets, kmax=kmax, nfrac=nfrac, amin=amin)[:2]
    
    for cnt, (ykey, idx) in enumerate(selected.iteritems()):
        
        subY = subYdict[ykey]
        #print ykey, subY
        
        subdistance = distance_matrix[idx][:, idx]
        
        # keys
        keylist.append(ykey)
        
        # pvalue
        pval = dbf_test(subdistance, subY)[0]
        pvals.append(pval)
        
        # classifier
        knn_parameters['n_neighbors'] = np.logspace(np.log10(1), np.log10(len(subY)/2), 10, dtype=int)
        knn_parameters['metric'] = ('precomputed',)
        grid = grid_search.GridSearchCV(knn, knn_parameters, cv=ncv, scoring=knn_error_metric)
        grid.fit(subdistance, subY)
        scores.append(1.0 - grid.best_score_) # see grid.grid_scores_, convert to error rate
        
        # compute baseline
        baseline.append(1.0 - 1.0*np.max(np.bincount(subY))/len(subY))
        
        print cnt, ykey, pvals[-1], scores[-1], baseline[-1]
        #if cnt>10:
            #break
    
    return np.array(keylist), np.array(pvals), np.array(scores), np.array(baseline)

In [134]:
def compute_and_plot(distancematrix, targets, outdir):
    #print len(inferencePrediction(X, targets))
    keylist, pvals, scores, baseline = inferencePrediction(distancematrix, targets)
    # save result to text
    
    outfile = os.path.join(outdir, "keylist.txt")
    np.savetxt(outfile, keylist, fmt='%s', delimiter='\t') 
    outfile = os.path.join(outdir, "pvals_errors.txt")
    np.savetxt(outfile, np.vstack((pvals, scores)).T, fmt=['%g', '%g'], delimiter='\t')
    
    # sort by average values
    sortidx = (scale(pvals)+scale(scores)).argsort()[::-1]
    bestkeys = keylist[sortidx]
    bestp = pvals[sortidx]
    bests = scores[sortidx]
    baseline = baseline[sortidx]
                                                           
    outfile = os.path.join(outdir, "bestkeylist.txt")
    np.savetxt(outfile, bestkeys, fmt='%s', delimiter='\t')
    outfile = os.path.join(outdir, "bestlist.txt")
    np.savetxt(outfile, np.vstack((bestp, bests, baseline)).T, fmt=['%g', '%g', '%g'], delimiter='\t')
    
    # save figure
    comb = {"pvalues": pd.Series(pvals, index=keylist),
            "error_rate":pd.Series(scores, index=keylist)}
    combined = pd.DataFrame(comb)
    
    outfile = os.path.join(outdir, "pvals_errors.pdf")
    sns.jointplot(x="pvalues", y="error_rate", data=combined, 
                  kind="reg", xlim=(0.0, 1.0), ylim=(0.0, 1.0))
    plt.savefig(outfile)
    
    return keylist, pvals, scores, baseline

In [103]:
outdir = os.path.join("C:\Users","Peter", "Desktop", "JSM2017", "Neuroscience","cutnorm","FOptM","FOptM-share", "results")


In [141]:
keylist, pvals, scores, baseline = compute_and_plot(distmatrix, targets, outdir)

0 sameevening:Howmuchdidtinnitusbotheryoutoday? 0.881901120424 0.7 0.5
1 panas:enthusiastic 0.46372490911 0.246153846154 0.338461538462
2 weight 0.0877830743869 0.625 0.6
3 sameevening:Timespentoutdoors 0.502718254943 0.478260869565 0.478260869565
4 panas:upset 0.754538890943 0.257142857143 0.257142857143
5 panas:proud 1.0 0.430769230769 0.476923076923
6 sameevening:Guthealth 1.0 0.559322033898 0.610169491525
7 panas:distressed 0.178942755336 0.257142857143 0.257142857143
8 panas:alert 0.0484771820113 0.220588235294 0.220588235294
9 sameevening:Stress 0.0352308568635 0.263157894737 0.333333333333
10 prevevening:Guthealth 0.352144579993 0.515625 0.609375
11 afterscan:Anxietyduringscan 0.694914783196 0.411764705882 0.455882352941
12 blood:eo 0.649768141812 0.333333333333 0.416666666667
13 afterscan:systolic 0.451435978111 0.666666666667 0.714285714286
14 panas:determined 0.184372691974 0.347826086957 0.36231884058
15 afterscan:diastolic 0.840330657559 0.842105263158 0.815789473684
16 ema

In [24]:
selected, subYdict = preprocessor(targets)[:2]


In [15]:
testy = selected['panas:strong']
testy1 = distmatrix[testy][:,testy]
print len(testy1[:,0])

68


In [25]:
subY = subYdict['panas:strong']

In [136]:
from sklearn.neighbors import KNeighborsClassifier


In [88]:
knn = KNeighborsClassifier(n_neighbors=22, metric = 'precomputed')

In [89]:
knn.fit(testy1,subY)
knn.score(testy1,subY)

0.55882352941176472

In [69]:
sum(subY == 1)*1.0/len(subY)

0.38235294117647056

In [81]:
knn_parameters['n_neighbors'] = np.logspace(np.log10(1), np.log10(len(subY)/2), 10, dtype=int)
knn_parameters['metric'] = ('precomputed',)

In [82]:
knn_parameters

{'metric': ('precomputed',),
 'n_neighbors': array([ 1,  1,  2,  3,  4,  7, 10, 15, 22, 34]),
 'p': (1,),
 'weights': ('uniform', 'distance')}

In [83]:
grid = grid_search.GridSearchCV(knn, knn_parameters, cv=5, scoring=knn_error_metric)
grid.fit(testy1, subY)
print(1.0 - grid.best_score_)

0.382352941176
